In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 22:54:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark.read.parquet("data/pq/green/*/*")

26/01/19 22:54:12 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/pq/green/*/*.
java.io.FileNotFoundException: File data/pq/green/*/* does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:980)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1301)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:970)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.sinks.FileStreamSink$.hasMetadata(FileStreamSink.scala:58)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:384)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.ResolveData

In [5]:
df_green.registerTempTable('green')

/opt/homebrew/Cellar/apache-spark/4.1.0/libexec/python/lib/pyspark.zip/pyspark/sql/classic/dataframe.py:178: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.


In [22]:
df_green_revenue = spark.sql("""
SELECT 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    SUM(total_amount) AS amount,
    COUNT(1) as number_records

FROM 
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
""")

In [7]:
df_green_revenue.show()

[Stage 2:============================================>              (6 + 2) / 8]

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
|   7|2020-01-01 00:00:00| 769.7299999999997|            45|
|  17|2020-01-01 00:00:00|195.03000000000006|             9|
|  18|2020-01-01 00:00:00|               7.8|             1|
|  22|2020-01-01 00:00:00|              15.8|             1|
|  24|2020-01-01 00:00:00|              87.6|             3|
|  25|2020-01-01 00:00:00| 531.0000000000001|            26|
|  29|2020-01-01 00:00:00|              61.3|             1|
|  32|2020-01-01 00:00:00| 68.94999999999999|             2|
|  33|2020-01-01 00:00:00|317.27000000000004|            11|
|  35|2020-01-01 00:00:00|            129.96|             5|
|  36|2020-01-01 00:00:00|            295.34|            11|
|  37|2020-01-01 00:00:00|            175.67|             6|
|  38|2020-01-01 00:00:00| 98.78999999999999|             2|
|  40|2020-01-01 00:00:0

In [27]:
df_green_revenue \
    .repartition(20) \
        .write.parquet('data/report/revenue/green', mode='overwrite')

26/01/19 23:24:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/01/19 23:24:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [20]:
print(spark.conf.get("spark.sql.adaptive.enabled"))
print(spark.conf.get("spark.sql.adaptive.coalescePartitions.enabled"))

true
true


In [9]:
df_yellow = spark.read.parquet("data/pq/yellow/*/*")

26/01/19 22:54:34 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/pq/yellow/*/*.
java.io.FileNotFoundException: File data/pq/yellow/*/* does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:980)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1301)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:970)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.sinks.FileStreamSink$.hasMetadata(FileStreamSink.scala:58)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:384)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.ResolveDa

In [10]:
df_yellow.registerTempTable('yellow')

In [23]:
df_yellow_revenue = spark.sql("""
SELECT 
    PULocationID AS zone,
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    SUM(total_amount) AS amount,
    COUNT(1) as number_records

FROM 
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
""")

In [28]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

26/01/19 23:25:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/01/19 23:25:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [29]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [30]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')   
                    

In [31]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'], how='outer')

In [32]:
df_join.show()

[Stage 51:========================>                                 (3 + 4) / 7]

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              NULL|                NULL|              25.0|                    1|
|2020-01-01 00:00:00|   4|              NULL|                NULL|1004.3000000000001|                   57|
|2020-01-01 00:00:00|  40|            168.98|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|              NULL|                NULL| 732.4800000000002|                   42|
|2020-01-01 00:00:00|  47|              13.3|                   1|               8.3|                    1|
|2020-01-01 00:00:00|  51|              17.8|                   2|              31.0|                    1|
|2020-01-01 00:00:00|  62|  

In [33]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

26/01/19 23:27:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [35]:
df_join= spark.read.parquet('data/report/revenue/total')

In [36]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  14|              NULL|                NULL|               8.8|                    1|
|2020-01-01 00:00:00|  15|              NULL|                NULL|             34.09|                    1|
|2020-01-01 00:00:00|  25| 531.0000000000001|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  59|50.900000000000006|                   3|              NULL|                 NULL|
|2020-01-01 00:00:00|  71|              23.8|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|  75| 278.1400000000001|                  26| 958.3500000000004|                   69|
|2020-01-01 00:00:00|  78|  

In [37]:
df_zones= spark.read.parquet('zones')

In [38]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [40]:
df_result=df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [42]:
df_result.drop('LocationID').drop('zone').write.parquet('tmp/revenue-zones')

26/01/19 23:42:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [44]:
df_result=df_result.drop('LocationID').drop('zone')

In [45]:
df_result.show()

+-------------------+------------------+--------------------+------------------+---------------------+-------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+-------+------------+
|2020-01-01 16:00:00|232.52000000000007|                  18|            137.44|                    5| Queens|   Boro Zone|
|2020-01-01 21:00:00|179.12000000000006|                  16|            103.21|                   10| Queens|   Boro Zone|
|2020-01-01 23:00:00|            255.93|                  14| 69.42000000000002|                    7| Queens|   Boro Zone|
|2020-01-02 13:00:00|372.07000000000016|                  28|            113.02|                   12| Queens|   Boro Zone|
|2020-01-02 15:00:00|199.95000000000007|                  22|            158.23|                   17| Queens|   Boro Zone|
|2020-01

26/01/20 00:23:51 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2031532 ms exceeds timeout 120000 ms
26/01/20 00:23:51 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/20 00:23:52 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$